[View in Colaboratory](https://colab.research.google.com/github/adityasingh1993/100DayMLChallenge/blob/master/NLP_Classification_Glove.ipynb)

In [1]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2018-10-23 18:47:09--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2018-10-23 18:47:10--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip.1’

glove.42B.300d.zip. 100%[===================>]   1.75G  3.05MB/s    in 6m 42s  

2018-10-23 18:53:51 (4.46 MB/s) - ‘glove.42B.300d.zip.1’ saved [1877800501/1877800501]



In [2]:
!unzip glove.42B.300d.zip

Archive:  glove.42B.300d.zip
replace glove.42B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [3]:
!wget https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-train-all-terms.txt

--2018-10-23 19:08:18--  https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-train-all-terms.txt
Resolving www.cs.umb.edu (www.cs.umb.edu)... 158.121.106.224
Connecting to www.cs.umb.edu (www.cs.umb.edu)|158.121.106.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3354424 (3.2M) [text/plain]
Saving to: ‘r8-train-all-terms.txt’

r8-train-all-terms. 100%[===================>]   3.20M  71.2KB/s    in 37s     

2018-10-23 19:08:56 (88.9 KB/s) - ‘r8-train-all-terms.txt’ saved [3354424/3354424]



In [4]:
!wget https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-test-all-terms.txt

--2018-10-23 19:09:03--  https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-test-all-terms.txt
Resolving www.cs.umb.edu (www.cs.umb.edu)... 158.121.106.224
Connecting to www.cs.umb.edu (www.cs.umb.edu)|158.121.106.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1195261 (1.1M) [text/plain]
Saving to: ‘r8-test-all-terms.txt’

r8-test-all-terms.t 100%[===================>]   1.14M   105KB/s    in 9.3s    

2018-10-23 19:09:13 (125 KB/s) - ‘r8-test-all-terms.txt’ saved [1195261/1195261]



In [0]:
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
import pandas as pd

In [8]:
print("Loading Word Vector")
word2vec={}
idx2word=[]
embedding=[]

with open('glove.42B.300d.txt') as f:
  for line in f:
    values=line.split()
    word=values[0]
    vec=np.asarray(values[1:],dtype='float32')
    word2vec[word]=vec
    embedding.append(vec)
    idx2word.append(word)
    
  print("found {} number of words".format(len(word2vec)))
  embedding=np.asarray(embedding)
  V,D=embedding.shape

Loading Word Vector
found 1917494 number of words


In [0]:
train=pd.read_csv('r8-train-all-terms.txt',header=None,sep='\t')
test=pd.read_csv('r8-test-all-terms.txt',header=None,sep='\t')

In [26]:
train.head(10)

,label,contents
0,earn,champion products ch approves stock split cham...
1,acq,computer terminal systems cpml completes sale ...
2,earn,cobanco inc cbco year net shr cts vs dlrs net ...
3,earn,am international inc am nd qtr jan oper shr lo...
4,earn,brown forman inc bfd th qtr net shr one dlr vs...
5,earn,dean foods df sees strong th qtr earnings dean...
6,earn,brown forman bfdb sets stock split ups payout ...
7,earn,esquire radio and electronics inc ee th qtr sh...
8,earn,united presidential corp upco th qtr net shr c...
9,earn,owens and minor inc obod raises qtly dividend ...


In [0]:
train.columns=['label','contents']
test.columns=['label','contents']

In [23]:
len(train)

5485

In [13]:
train.head(2)

,label,contents
0,earn,champion products ch approves stock split cham...
1,acq,computer terminal systems cpml completes sale ...


In [0]:
def transform_Data(data):
  X=np.zeros((len(data),D))
  n=0
  empty=0
  for sentence in data:
    token=sentence.split()
    vecs=[]
    for word in token:
      if word in word2vec:
        vec=word2vec[word]
        vecs.append(vec)
    if len(vecs)>0:
      vecs=np.array(vecs)
      X[n]=vecs.mean(axis=0)
    else:
      empty+=1
    n+=1
  print("{} is the number of word not found".format(empty))
  return X
      

In [24]:
Xtrain=transform_Data(train['contents'])
Ytrain=train.label

0 is the number of word not found


In [22]:
len(X)

5485

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
rf=RandomForestClassifier()

In [29]:
rf.fit(Xtrain,Ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [31]:
print(rf.score(Xtrain,Ytrain))

0.9972652689152234


In [33]:
Xtest=transform_Data(test['contents'])
Ytest=test.label

0 is the number of word not found


In [34]:
print(rf.score(Xtest,Ytest))

0.9013248058474189
